In [ ]:
# -*- coding: utf-8 -*-
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or
# implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#

# Text processing in Python (continued)
- [NLTK](http://www.nltk.org/book/)
- gensim
- word2vec
- scikit-learn

In [ ]:
import nltk
from nltk.corpus import inaugural

In [ ]:
inaugural.fileids()

In [ ]:
inaugural.words('1789-Washington.txt')

In [ ]:
len(inaugural.words('1789-Washington.txt'))

In [ ]:
inaugural.sents('1789-Washington.txt')

In [ ]:
texts = []
for fileid in inaugural.fileids():
    year = fileid[:4]
    name = fileid[5:].split('.')[0]
    text = ' '.join(inaugural.words(fileid))
    texts.append({'name': name, 'year': year, 'text': text})

In [ ]:
texts[0]

## Tokenization

In [ ]:
text = texts[0]['text']

In [ ]:
sentences = nltk.sent_tokenize(text)

In [ ]:
sentences[:5]

In [ ]:
sent = sentences[0]

In [ ]:
tokens = nltk.word_tokenize(sent)

In [ ]:
tokens

## Stemming

Stemming returns the roots of words. Example in Slovak: *ryba -> ryb*.

In [ ]:
porter = nltk.PorterStemmer()

In [ ]:
[porter.stem(token) for token in tokens]

## Lemmatization

Lemmatization converts words to their basic dictionary form. Example in Slovk: *rybe -> ryba*.

In [ ]:
wnl = nltk.WordNetLemmatizer()

In [ ]:
[wnl.lemmatize(token) for token in tokens]

## Part-of-Speech Tagging (POS)

In [ ]:
tagged = nltk.pos_tag(tokens)
tagged

In [ ]:
nltk.help.upenn_tagset('IN')

In [ ]:
nltk.help.upenn_tagset('NNP')

## Name entities

In [ ]:
entities = nltk.chunk.ne_chunk(tagged)

In [ ]:
print(entities.__repr__())

## N-grams

In [ ]:
tokens = nltk.word_tokenize(text)

In [ ]:
bigrams = list(nltk.bigrams(tokens))
bigrams[:5]

In [ ]:
nltk.FreqDist(bigrams).most_common(10)

In [ ]:
stopwords = nltk.corpus.stopwords.words('english')

In [ ]:
stopwords[:10]

In [ ]:
tokens_cleared = [token.lower() for token in tokens if token.isalpha() and token.lower() not in stopwords]

In [ ]:
tokens_cleared[:10]

In [ ]:
nltk.FreqDist(nltk.bigrams(tokens_cleared)).most_common(10)

In [ ]:
nltk.FreqDist(nltk.trigrams(tokens_cleared)).most_common(10)

## WordNet

* Lexical database
* Contains synsets: nouns, verbs, adjectives, adverbs
* Connections between synsets: antonyms, hyperonyms, hyponyms, holonyms, meronyms

In [ ]:
from nltk.corpus import wordnet as wn

In [ ]:
wn.synsets('car')

In [ ]:
car = wn.synset('car.n.01')

In [ ]:
car.lemma_names()

In [ ]:
car.definition()

In [ ]:
car.examples()

In [ ]:
car.hyponyms()

In [ ]:
car.hypernyms()

In [ ]:
car.part_meronyms()

In [ ]:
wn.synsets('black')[0].lemmas()[0].antonyms()

# Text Representation

A text document is usually represented using a bag-of-words (BoW) model, which is a vector. The components of the vector represent individual words or n-grams from a dictionary (for the entire corpus/language). The values ​​of the vector components can be:

* count
* frequency
* weighted frequency

Words with high frequency in a language (such as conjunctions) are referred to as *stop words* and are often removed during preprocessing.

## TF-IDF
* Term frequency * inverse document frequency
* `TF` – frequency of a word in the current document
* `IDF` – negative logarithm of the probability of a word occurring in a document (same for all documents)
* Various variants (weighting schemes): https://en.wikipedia.org/wiki/Tf%E2%80%93idf

## Gensim
- A library for modeling topics in documents.
- Implements TF-IDF, LSA, pLSA, LDA, HDP, DTM, word2vec
- https://radimrehurek.com/gensim/tutorial.html

In [ ]:
from gensim import corpora, models, similarities

In [ ]:
def preprocess_text(text):
    tokens = nltk.word_tokenize(text)
    stopwords = nltk.corpus.stopwords.words('english')
    return [token.lower() for token in tokens if token.isalpha() and token.lower() not in stopwords]

In [ ]:
tokenized_docs = [preprocess_text(text['text']) for text in texts]

In [ ]:
tokenized_docs[4][:10]

Removing words that occur only once in the corpus

In [ ]:
from collections import defaultdict

frequency = defaultdict(int)
for text in tokenized_docs:
    for token in text:
        frequency[token] += 1

tokenized_docs = [[token for token in doc if frequency[token] > 1] for doc in tokenized_docs]

In [ ]:
dictionary = corpora.Dictionary(tokenized_docs)

In [ ]:
# print(dictionary.token2id)

In [ ]:
corpus = [dictionary.doc2bow(doc) for doc in tokenized_docs]

Training the TF-IDF model

In [ ]:
tfidf_model = models.TfidfModel(corpus)

In [ ]:
tfidf_corpus = tfidf_model[corpus]

In [ ]:
tfidf_corpus[0][:10]

Other models: LSI, LDA, ...

We can calculate the similarity of the resulting vectors:

In [ ]:
index = similarities.MatrixSimilarity(tfidf_corpus)

In [ ]:
index[tfidf_corpus[0]]

## word2vec

Each word has a learned vector of real numbers that represent its various properties and capture several linguistic regularities. We can count the similarity between words as the similarity of two vectors.

vector('Paris') - vector('France') + vector('Italy') ~= vector('Rome')

vector('king') - vector('man') + vector('woman') ~= vector('queen')

https://radimrehurek.com/gensim/models/word2vec.html

https://medium.com/@mishra.thedeepak/word2vec-in-minutes-gensim-nlp-python-6940f4e00980

In [ ]:
from nltk.corpus import brown

In [ ]:
sentences = brown.sents()
model = models.Word2Vec(sentences, min_count=1)

In [ ]:
model.save('brown_model')

In [ ]:
model = models.Word2Vec.load('brown_model')

In [ ]:
print(model.wv.most_similar("mother"))

In [ ]:
print(model.wv.doesnt_match("breakfast cereal dinner lunch".split()))

In [ ]:
print(model.wv.doesnt_match("pizza pasta garden fries".split()))

In [ ]:
model.wv['human']

## Feature extraction using scikit-learn

https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
docs = [text['text'] for text in texts]

In [ ]:
vectorizer = CountVectorizer(stop_words='english')
tf = vectorizer.fit_transform(docs)

In [ ]:
tf.toarray()[0][:100]

In [ ]:
vectorizer.get_feature_names()[:100]

In [ ]:
transformer = TfidfVectorizer(stop_words='english')
tfidf = transformer.fit_transform(docs)

In [ ]:
tfidf.toarray()[0][:100]